In [1]:
%pip install qiskit==1.2.4
%pip install qiskit-aer==0.15.1
%pip install pylatexenc==2.10


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_gate
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Operator
from qiskit.quantum_info import Statevector
from qiskit import transpile 
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import ControlledGate
import math 
import numpy as np

from qiskit.visualization import plot_bloch_multivector
from qiskit.quantum_info import Statevector
# The aim of the assignment is to simulate the Ekert91 key distribution protocol.

# This notebook is for a simulation of the protocol without an attacker.
U_matrix = np.array([[1/np.sqrt(3), -np.sqrt(2)/np.sqrt(3)],
                     [np.sqrt(2)/np.sqrt(3), 1/np.sqrt(3)]])
U_operator = Operator(U_matrix)

root2 = math.sqrt(2)
denom1 = math.sqrt(4 + 2*root2)
denom2 = math.sqrt(4 - 2*root2) 

# Define W transform matrix
w_matrix = [ [ -1 / denom1 , (1 + root2) / denom1 ],
            [  1 / denom2 , (root2 - 1) / denom2 ] ]

v_matrix = [ [  1 / denom1 , (1 + root2) / denom1 ],
            [ -1 / denom2 , (root2 - 1) / denom2 ] ]

w_operator = Operator(w_matrix)
v_operator = Operator(v_matrix)

def probability():
    q = QuantumCircuit(1,1)
    q.append(U_operator,[0])
    q.measure(0,0)

    backend = BasicSimulator()
    compiled = transpile(q, backend)
    job_sim = backend.run(compiled, shots=1)
    result_sim = job_sim.result()
    counts = result_sim.get_counts(compiled)
    return int(list(counts.keys())[0])

def random():
    circuit = QuantumCircuit(1,1)
    circuit.h(0)  # apply H to put it in equal superposition state
    circuit.measure(0,0)

    backend = BasicSimulator()
    compiled = transpile(circuit, backend)
    job_sim = backend.run(compiled, shots=1)
    result_sim = job_sim.result()
    counts = result_sim.get_counts(compiled)
    return int(list(counts.keys())[0]) # 0 or 1 with equal probability

def entangledPair():
    q = QuantumCircuit(2, 2) 
    q.h(0)        # Step 1: Create superposition on qubit 0
    q.cx(0, 1)    # Step 2: Apply CNOT to entangle (|00⟩ + |11⟩) / sqrt(2)
    q.x(1)        # Step 3: Apply X to qubit 1 to flip state to (|01⟩ - |10⟩) / sqrt(2)
    q.z(1)
    return q


def alice_bases():
    if probability() == 0:
        return "X"
    else:
        choice = random()
        if choice == 0:
            return "W"
        else:
            return "Z"

def bob_bases():
    if probability() == 0:
        return "W"
    else:
        choice = random()
        if choice == 0:
            return "Z"
        else:
            return "V"

def apply_measurement(qc, qubit, basis):
    if basis == "X":
        qc.h(qubit)
    elif basis == "W":
        qc.append(w_operator, [qubit])
        # qc.h(qubit)
    elif basis == "V":
        qc.append(v_operator, [qubit])
        # qc.h(qubit)
    # Z basis doesnt require transformation
    qc.measure([qubit], [qubit])

def convert(num):
    if num == 1:
        num = -1
    elif num == 0:
        num = 1
    return num

N = 100
key = []
S = 0
M = int(9*N/2)

# alice_key = []
# bob_key = []

bell_test_results = {"XW": [], "XV": [], "ZW": [], "ZV": []}

for i in range(M):
    qc = entangledPair()
    a_basis = alice_bases()
    b_basis = bob_bases()
    apply_measurement(qc, 1, a_basis)
    apply_measurement(qc, 0, b_basis)
    
    # simulate circuit
    backend = BasicSimulator()
    compiled = transpile(qc, backend)
    job_sim = backend.run(compiled, shots=1)
    result_sim = job_sim.result()
    counts = result_sim.get_counts(compiled)
    measurement = list(counts.keys())[0]
    alice_result, bob_result = int(measurement[0]), int(measurement[1])  # Separate bits
    if (a_basis, b_basis) in [("W","W"), ("Z","Z")] :
        key.append(alice_result)
        bob_result ^= 1 # flip bob's bits
        # alice_key.append(alice_result)
        # bob_key.append(bob_result ^ 1) 
    elif (a_basis, b_basis) == ("X","W"):
        bell_test_results["XW"].append(convert(alice_result) * convert(bob_result))
    elif (a_basis, b_basis) == ("Z","W"):
        bell_test_results["ZW"].append(convert(alice_result) * convert(bob_result))
    elif (a_basis, b_basis) == ("Z","V"):
        bell_test_results["ZV"].append(convert(alice_result) * convert(bob_result))
    elif (a_basis, b_basis) == ("X","V"):
        bell_test_results["XV"].append(convert(alice_result) * convert(bob_result))

XW_avg = sum(bell_test_results["XW"])/len(bell_test_results["XW"])
ZW_avg = sum(bell_test_results["ZW"])/len(bell_test_results["ZW"])
ZV_avg = sum(bell_test_results["ZV"])/len(bell_test_results["ZV"])
XV_avg = sum(bell_test_results["XV"])/len(bell_test_results["XV"])

S = (XW_avg - XV_avg + ZW_avg + ZV_avg)
# print("Alice's key:", alice_key)
# print("Bob's key:", bob_key)
# print(alice_key == bob_key)
print("S:", abs(S))

S: 2.5096023416863
